In [1]:
#    X    # Won      : Total number of wins 
#    X    # Lost     : Total number of losses 
#====V====# Pct      : Percent of wins 

#====V====# Homewon  : Number of games won at home 
#====V====# Homelost : Number of games lost at home 

#====V====# Roadwon  : Number of games won away 
#====V====# Roadlost : Number of games lost away 

#    X    # Divwon   : Number of games won in division 
#    X    # Divlost  : Number of games lost in division 

#    X    # Confwon  : Number of games won in conference 
#    X    # Conflost : Number of games lost in conference 

#    X    # Streak   : Number of consecutive wins/losses 

#====V====# L10won   : Number of wins in last 10 games 
#====V====# L10lost  : Number of losses in last 10 games 




In [2]:
import numpy as np 
import pandas as pd

C:\Users\User\AppData\Local\Temp\ipykernel_11844\118197663.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
games = pd.read_csv("../Data/volley.csv")
games['combined_date'] = games['year'].astype(str) + '-' + games['month'].astype(str) + '-' + games['Date'].astype(str)
games['combined_date'] = pd.to_datetime(games['combined_date'])


games["home_Pct"] = games["home_win_percentage"]
games["away_Pct"] = games["away_win_percentage"]

games["home_form"] =  games["home_team_form"]
games["away_form"] =  games["away_team_form"]


drop_columns = ["Date", "month", "year",
                "home_win_percentage","away_win_percentage",
                "home_team_form","away_team_form"]
games.drop(columns=drop_columns, inplace=True)

games.head()

,away_last_yr_pos,away_prev_game_perf,away_team_av_points,away_team_av_points_conceded,away_team_away_form,away_team_rest_time,h2h_form,home_current_pos,home_last_yr_pos,home_prev_game_perf,...,home_team_rest_time,match_importance,result,away_team,home_team,combined_date,home_Pct,away_Pct,home_form,away_form
12,12,0,20.0,20.00,0.0,7,0.0,2,2,0,...,7,0,1,Santo Andre,America Volei,2010-11-07,50.0,50.0,0.0,0.0
7,6,0,20.0,20.00,0.0,7,0.0,8,1,0,...,7,0,1,Fatima UCS,Cimed,2010-11-09,50.0,50.0,0.0,0.0
13,3,0,20.0,20.00,0.0,7,0.0,11,5,0,...,7,0,1,Sada Cruzeiro,Sesi,2010-11-11,50.0,50.0,0.0,0.0
15,8,0,20.0,20.00,0.0,7,0.0,11,4,0,...,7,0,1,Minas,Pinheiros,2010-11-11,50.0,50.0,0.0,0.0
13,3,0,20.4,20.64,-0.2,2,0.0,3,4,1,...,2,0,0,Sada Cruzeiro,Pinheiros,2010-11-13,100.0,0.0,0.4,-0.2


In [4]:
# games.info()
games.columns

Index(['away_last_yr_pos', 'away_prev_game_perf', 'away_team_av_points',
       'away_team_av_points_conceded', 'away_team_away_form',
       'away_team_rest_time', 'h2h_form', 'home_current_pos',
       'home_last_yr_pos', 'home_prev_game_perf', 'home_team_av_points',
       'home_team_av_points_conceded', 'home_team_home_form',
       'home_team_rest_time', 'match_importance', 'result', 'away_team',
       'home_team', 'combined_date', 'home_Pct', 'away_Pct', 'home_form',
       'away_form'],
      dtype='object')

In [5]:
game_copy = games.copy()

In [6]:
# ========= HOME =========
game_copy["Homewon"] = game_copy.groupby(["home_team"])["result"].cumsum()
game_copy["Homewon"] = game_copy.groupby(["home_team"])["result"].shift(1)

game_copy["nbr_match"] =  game_copy.groupby(["home_team"])["result"].cumcount()
game_copy["Homelost"] = game_copy["nbr_match"] - game_copy["Homewon"]
game_copy  = game_copy.drop(columns=[ "nbr_match"])

# ========= AWAY =========
game_copy["Roadwon"] = game_copy.groupby(["away_team"])["result"].cumsum()
game_copy["Roadwon"] = game_copy.groupby(["away_team"])["result"].shift(1)

game_copy["nbr_match"] =  game_copy.groupby(["away_team"])["result"].cumcount()
game_copy["Roadlost"] = game_copy["nbr_match"] - game_copy["Roadwon"]
game_copy  = game_copy.drop(columns=[ "nbr_match"])


In [7]:
columns_to_have = [
                   "home_Pct",
                   "away_Pct",

                   "home_form",
                   "away_form",


                   "Roadwon",
                   "Roadlost",
                   "Homewon",
                   "Homelost",

                   "result",
                   "away_team",
                   "home_team",
                   ]
final_game_csv = game_copy[columns_to_have]


In [8]:
final_game_csv = final_game_csv.dropna(subset=columns_to_have)
final_game_csv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1235 entries, 13 to 2
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   home_Pct   1235 non-null   float64
 1   away_Pct   1235 non-null   float64
 2   home_form  1235 non-null   float64
 3   away_form  1235 non-null   float64
 4   Roadwon    1235 non-null   float64
 5   Roadlost   1235 non-null   float64
 6   Homewon    1235 non-null   float64
 7   Homelost   1235 non-null   float64
 8   result     1235 non-null   int64  
 9   away_team  1235 non-null   object 
 10  home_team  1235 non-null   object 
dtypes: float64(8), int64(1), object(2)
memory usage: 115.8+ KB


In [9]:
# File path where you want to save the CSV
file_path = "../Data/volley_for_nba_prepro.csv"

# Save DataFrame to CSV
final_game_csv.to_csv(file_path, index=False)